# Importamos Librerias

In [1]:
from utils import *

In [2]:
# Cargamos los metadatos de CoinGecko
exchanges_metadata = read_json("./metadata/coingecko/exchanges.json")
coins_metadata = read_json("./metadata/coingecko/coins.json")
categories_metadata = read_json("./metadata/coingecko/categories.json")

## Obtener la info de las coins de las categorias principales

Obtener las categorias principales (IA, RWA, )

In [3]:
# Filtrar las categorías utilizando la expresión regular
regex_categories = create_regex(
    words=["meme", "memes", "game", "gaming", "games", "ai", "real world assets", "rwa", "juego"]
)

# Filtrar las categorías que contienen las palabras clave
categories_filter = list(
    filter(
        lambda categoria: regex_categories.search(categoria["name"].lower()),
        categories_metadata,
    )
)

Ahora realizamos las peticiones a la api para obtener la info de las coins de las categorias principales

In [7]:
coins_info: list[dict] = [] # Lista de diccionarios con la información de las monedas
for category in categories_filter:
    page = 1 # Página de la API
    coin_info:list[dict] = [] # Lista de diccionarios con la información de las monedas
    # Usamos un bucle infinito para obtener todas las monedas de la categoría
    while True:
        try:
            response = fetch_data(api=APIS.COINGECKO,uri=f"coins/markets?vs_currency=usd&category={category["category_id"]}&page={page}",parameters={})
        except:
            break
        coin_info.extend(response) # Añadimos la información de las monedas a la lista
        page += 1 # Incrementamos la página
    category_new = category.copy() # Copiamos la categoría
    category_new["coins"] = coin_info # Añadimos la información de las monedas a la categoría
    coins_info.append(category_new) # Añadimos la información de la categoría a la lista
    time.sleep(2) # Esperamos 2 segundos

In [8]:
# Calcular el total de monedas
total_coins = 0
for coin in coins_info:
    total_coins += len(coin["coins"])
total_coins

1851

Estandarizamos los coins

In [9]:
new_coins_info:list[dict] = [] # Lista de diccionarios que contiene la información de las monedas
for category in coins_info:
    for coin in category["coins"]:
        new_coin = coin.copy() # Copiar el diccionario de la moneda
        new_coin["category"] = category["name"] # Agregar el nombre de la categoría
        new_coin["category_id"] = category["category_id"]  # Agregar el id de la categoría
        new_coins_info.append(new_coin) # Agregar el diccionario de la moneda a la lista

In [10]:
len(new_coins_info)

1851

In [11]:
# Guardamos la información en un archivo JSON
save_json(new_coins_info, f"./data/json/coingecko/coins.json")

## Obtener la info de las tickers de los exchanges principales 

In [12]:
len(coins_metadata)

14024

La API de CoinGecko ya nos devuelve de la exchange mas importante a la menos importante

In [13]:
# Obtendremos los datos de los 20 principales exchanges
exchanges_centralized_info = []
exchanges_decentralized_info = []

for index, exchange in enumerate(exchanges_metadata):
    # Verificamos que tengamos 20 exchanges centralizados y 20 exchanges descentralizados
    if len(exchanges_centralized_info) > 20 and len(exchanges_decentralized_info) > 20:
        break
    
    # Obtenemos la información del exchange
    exchange_info = fetch_data(api= APIS.COINGECKO, uri=f"exchanges/{exchange["id"]}", parameters={})
    
    # Agregamos el id del exchange
    exchange_info["id"] = exchange["id"]
    
    # Verificamos si el exchange es centralizado
    if (exchange_info["centralized"] and len(exchanges_centralized_info) < 21):
        exchanges_centralized_info.append(exchange_info) # Agregamos la información del exchange a la lista de exchanges centralizados
        
    # Verificamos si el exchange es descentralizado
    elif (not exchange_info["centralized"] and len(exchanges_decentralized_info) < 21):
        exchanges_decentralized_info.append(exchange_info)
    time.sleep(2)

Realizamos las peticiones a la API para obtener la info de cada moneda

In [14]:
# Obtner informacion de las transacciones entre monedas que suceden
def get_tickers_by_exchange(exchanges: list[dict]) -> list[dict]:
    exchange_tickers_info: list[dict] = [] # Lista para almacenar la información de los tickers de los exchanges
    for exchange in exchanges:
        page = 1 # variable para la paginacion de la API
        exchange_ticker_info = [] # Lista para almacenar la información de los tickers del exchange
        # Usamos un while para obtener toda la información de los tickers del exchange de cada paginacion 
        while True:
            try:
                response = fetch_data(api= APIS.COINGECKO, uri=f"exchanges/{exchange["id"]}/tickers?page={page}", parameters={})
            except:
                break
            if not response["tickers"]: break # Si no hay tickers en la respuesta, salimos del bucle
            exchange_ticker_info.extend(response["tickers"]) # Agregamos la información de los tickers a la lista
            page += 1 # Incrementamos la variable de paginacion
            time.sleep(2) # Esperamos 2 segundos para hacer la siguiente peticion
        exchange_new = exchange.copy() # Copiamos la información del exchange
        exchange_new["tickers"] = exchange_ticker_info # Agregamos la información de los tickers al exchange
        exchange_tickers_info.append(exchange_new) # Agregamos la información del exchange a la lista de exchanges
    return exchange_tickers_info

In [17]:
# LLamamos a la funcion para obtener la informacion de los tickers de los exchanges
exchanges_centralized_info = get_tickers_by_exchange(exchanges_centralized_info) 
exchanges_decentralized_info = get_tickers_by_exchange(exchanges_decentralized_info) 

## Obtener la info de las coins en base a los exchanges principales 

Creamos un set de cada exchange de la coins que maneja

In [18]:
coins_id = set() # Conjunto para almacenar los id de las monedas
for category in coins_info: # Recorremos las categorias
    for coin in category["coins"]: # Recorremos las monedas
        coins_id.add(coin["id"]) # Agregamos el id de la moneda al conjunto

Creamos una funcion que añadira el nuevo atributo `coins` a las exchanges

In [19]:
def get_coins_by_exchange(
    exchanges_info: list[dict], filter_by_category: bool = True
) -> list[dict]:
    new_exchange_info = [] # Lista para almacenar la información de los exchanges
    for exchange in exchanges_info: # Recorremos la información de los exchanges
        new_exchange = exchange.copy() # Copiamos la información del exchange
        new_coins = [] # Lista para almacenar los id de las monedas
        for coin in exchange["tickers"]: # Recorremos los tickers del exchange
            if (coin["coin_id"] in coins_id) or not filter_by_category: # Verificamos si el id de la moneda esta en el conjunto de id de monedas
                new_coins.append(coin["coin_id"]) # Agregamos el id de la moneda a la lista
        new_exchange["coins_id"] = new_coins # Agregamos la lista de id de las monedas al exchange
        new_exchange_info.append(new_exchange) # Agregamos la información del exchange a la lista
    return new_exchange_info

In [24]:
# Llamamos a la función para obtener la información de las monedas de los exchanges
new_exchange_centralized_info = get_coins_by_exchange(exchanges_centralized_info, False)
new_exchange_decentralized_info = get_coins_by_exchange(exchanges_decentralized_info, False)

In [21]:
# Guardamos la información en un archivo JSON
save_json(new_exchange_centralized_info, f"./data/json/coingecko/exchanges_centralized_coins.json")
save_json(new_exchange_decentralized_info, f"./data/json/coingecko/exchanges_decentralized_coins.json")

In [ ]:
for exchange in new_exchange_centralized_info:
    print(exchange["name"], len(exchange["coins_id"]))

for exchange in new_exchange_decentralized_info:
    print(exchange["name"], len(exchange["coins_id"]))

In [ ]:
#TODO Unir ciertas exchanges que estan repetidas con diferentes versiones y algo mas